In [1]:
import numpy as np
import pandas as pd
import torch
import random

from tqdm import tqdm
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
batch_size = 8
learning_rate = 1e-3
n_epochs = 1

random_seed = int(random.random() * 100)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
train_df = pd.read_csv('train_kdd_small.csv').to_numpy()
test_df = pd.read_csv('test_kdd_small.csv').to_numpy()
x_train = train_df[:, :-1]
y_train = train_df[:, -1]
x_test = test_df[:, :-1]
y_test = test_df[:, -1]

for col in range(1, 4):
    x_train[:, col] = LabelEncoder().fit_transform(x_train[:, col])
    x_test[:, col] = LabelEncoder().fit_transform(x_test[:, col])

x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)
y_train = LabelEncoder().fit_transform(y_train)
y_test = LabelEncoder().fit_transform(y_test)

x_train = torch.from_numpy(x_train.astype(np.float32)).to(device)
x_test = torch.from_numpy(x_test.astype(np.float32)).to(device)
y_train = torch.from_numpy(y_train.astype(np.int64)).to(device)
y_test = torch.from_numpy(y_test.astype(np.int64))

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()        
        self.flatten = nn.Flatten()
        self.network = nn.Sequential(
            nn.Linear(x_train.shape[1], 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 8),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(8, 2),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        pred = self.network(x)
        
        return pred

In [5]:
def train_loop(n_epochs, model, loss_fn, optimizer, scheduler):
    model.train()
    for epoch in tqdm(range(n_epochs)):
        for xb, yb in train_dl:
            optimizer.zero_grad()
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            optimizer.step()
            scheduler.step()
        
        if epoch % (n_epochs / 10) == 0:
            print('loss:', loss.item())
            print('lr:', scheduler.get_last_lr())

In [6]:
model = Net().to(device)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=learning_rate, epochs=n_epochs, steps_per_epoch=len(train_dl))
#scheduler = lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=1)
loss_fn = nn.CrossEntropyLoss()

train_loop(n_epochs, model, loss_fn, optimizer, scheduler)

100%|██████████| 1/1 [00:00<00:00,  1.74it/s]

0 loss = 0.31631380319595337
lr: [4.510549641772479e-08]


In [7]:
model.eval()
with torch.no_grad():
    y_pred = model(x_test).argmax(dim=1).cpu()
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('F1-measure: %.3f' % f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Confusion Matrix:')
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))

Accuracy: 1.00
Precision: 1.000
Recall: 1.000
F1-measure: 1.000
Confusion Matrix:
[[600   0]
 [  0 599]]
